In [11]:
## Задание 1.

import requests
from datetime import datetime, timedelta

In [12]:
# Получаем данные за вчера
yesterday = datetime.now() - timedelta(1)
yesterday_str = yesterday.strftime('%Y-%m-%d')
# записываем API-ключ
api_key = '09a1b80a4f56e541ac8d39156b4ca42d'
# URL для запроса курса валют
url = f'http://data.fixer.io/api/{yesterday_str}?access_key={api_key}&symbols=USD,EUR'
# Выполняем запрос к API
response = requests.get(url)
# проверяем статус и текст вывода
print(response.status_code)
print(response.text)

200
{"success":true,"timestamp":1715558399,"historical":true,"base":"EUR","date":"2024-05-12","rates":{"USD":1.077017,"EUR":1}}


In [ ]:
# Если запрос успешен
if response.status_code == 200:
    data = response.json()

    # Если в ответе есть ошибка
    if 'error' in data:
        with open('error_log.txt', 'a') as file:
            file.write(f"{yesterday_str}: {data['error']}\n")
    else:
        # Получаем курс валют
        rate = data['rates']['EUR'] / data['rates']['USD']

        # Подключаемся к БД
        connection = psycopg2.connect(
        database="db_name",
        user="bukhanastyuk_ed",
        password="password",
        host="host",
        port= 6432
    )
        # Создаем курсор для выполнения SQL-запросов
        cur = connection.cursor()

        # Проверяем, есть ли уже запись на эту дату
        cur.execute("SELECT * FROM exchange_rates WHERE date = %s AND pair = 'USD/EUR'", (yesterday,))
        if cur.fetchone() is None:
            # Если записи нет, вставляем данные
            cur.execute("INSERT INTO exchange_rates (date, pair, rate) VALUES (%s, 'USD/EUR', %s)", (yesterday, rate))
            conn.commit()

        cur.close()
        conn.close()
else:
    with open('error_log.txt', 'a') as file:
        file.write(f"{yesterday_str}: Request failed with status code {response.status_code}\n")

In [2]:
## Задание 2.
# напишем функцию для подсчета количества друзей в списке
def count_friends(lists):
    friend_count = {} # создадим словарь: ключ - id пользователя, а значение - количество его друзей
    for lst in lists:
        for id in lst:
            if id in friend_count: # если id есть в словаре
                friend_count[id] += len(lst) - 1 #увеличиваем кол-во друзей на 1 (кол-во элементов списка -1)
            else: # если id нет в словаре
                friend_count[id] = len(lst) - 1 # добавляем в словарь и увеличиваем кол-во друзей на 1 (кол-во элементов списка -1), если друзей нет, то будет 0
    return friend_count

a = [[2,3],[3,4],[5],[2,6],[2,4],[6,1]]
b = [[11,22],[33],[88],[66,22]]
print(count_friends(a))
print(count_friends(b))


{2: 3, 3: 2, 4: 2, 5: 0, 6: 2, 1: 1}
{11: 1, 22: 2, 33: 0, 88: 0, 66: 1}
